In [2]:
import pandas as pd

In [9]:
pip install category_encoders

     |████████████████████████████████| 102kB 1.4MB/s 


In [2]:
df = pd.read_csv('Downloads/berlin-airbnb-data/listings_summary.csv', engine='python', encoding='utf-8', error_bad_lines=False)
df.shape

(22552, 96)

In [ ]:
columns = [
    'neighbourhood_group_cleansed',
    'room_type',
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'bed_type',
    'amenities',
    'price',
    'security_deposit',
    'cleaning_fee',
    'minimum_nights',
    'instant_bookable'         
]

df = df[columns]

In [4]:
df['bathrooms'] = df['bathrooms'].fillna(df['bathrooms'].mean()).astype(float)
df['bedrooms'] = df['bedrooms'].fillna(df['bedrooms'].mean()).astype(float)
df['beds'] = df['beds'].fillna(df['beds'].mean()).astype(float)
df['security_deposit'] = df['security_deposit'].fillna('$0.00')
df['cleaning_fee'] = df['cleaning_fee'].fillna('$0.00')

df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)
df['security_deposit'] = df['security_deposit'].str.replace('$', '').str.replace(',', '').astype(float)
df['cleaning_fee'] = df['cleaning_fee'].str.replace('$', '').str.replace(',', '').astype(float)

df['neighborhood'] = df['neighbourhood_group_cleansed']
df = df.drop(columns=['neighbourhood_group_cleansed'])
df.isna().sum()

room_type           0
accommodates        0
bathrooms           0
bedrooms            0
beds                0
bed_type            0
amenities           0
price               0
security_deposit    0
cleaning_fee        0
minimum_nights      0
instant_bookable    0
neighborhood        0
dtype: int64

In [5]:
df['instant_bookable'] = df['instant_bookable'].replace({'t': 1, 'f': 0})
df['neighborhood'] = df['neighborhood'].str.replace('ö', 'o')
df.head(4)

,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable,neighborhood
0,Entire home/apt,3,1.0,1.0,2.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,Gym,Heating,""Famil...",60.0,200.0,30.0,4,0,Mitte
1,Private room,2,1.0,1.0,1.0,Real Bed,"{Wifi,Kitchen,Elevator,Heating,Washer,Essentia...",17.0,0.0,0.0,2,0,Pankow
2,Entire home/apt,4,1.0,1.0,2.0,Real Bed,"{Internet,Wifi,Kitchen,""Buzzer/wireless interc...",90.0,200.0,50.0,62,1,Pankow
3,Private room,2,1.0,1.0,1.0,Pull-out Sofa,"{Internet,Wifi,""Pets allowed"",""Pets live on th...",26.0,250.0,30.0,5,0,Tempelhof - Schoneberg


In [6]:
df['amenities'] = df['amenities'].str.replace('{',"[")
df['amenities'] = df['amenities'].str.replace('}',"]")
df['amenities'] = df['amenities'].str.replace('"',"")
df['amenities'] = df['amenities'].apply(lambda x: x.lower())

df['wifi'] = df['amenities'].str.contains('wifi').astype(str)
df['wifi'] = df['wifi'].replace({'True': 1, 'False': 0})
df = df.drop(columns='amenities')


df.head()

,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable,neighborhood,wifi
0,Entire home/apt,3,1.0,1.0,2.0,Real Bed,60.0,200.0,30.0,4,0,Mitte,1
1,Private room,2,1.0,1.0,1.0,Real Bed,17.0,0.0,0.0,2,0,Pankow,1
2,Entire home/apt,4,1.0,1.0,2.0,Real Bed,90.0,200.0,50.0,62,1,Pankow,1
3,Private room,2,1.0,1.0,1.0,Pull-out Sofa,26.0,250.0,30.0,5,0,Tempelhof - Schoneberg,1
4,Private room,2,1.0,1.0,2.0,Real Bed,42.0,0.0,0.0,2,0,Pankow,1


In [7]:
df.describe()

,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,minimum_nights,instant_bookable,wifi
count,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000,22552.000000
mean,2.637460,1.092695,1.161134,1.620558,67.143668,118.284454,18.253991,7.157059,0.311901,0.954106
std,1.510548,0.329660,0.645898,1.173798,220.266210,304.232661,28.172816,40.665073,0.463280,0.209260
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,1.000000,30.000000,0.000000,0.000000,2.000000,0.000000,1.000000
50%,2.000000,1.000000,1.000000,1.000000,45.000000,0.000000,10.000000,2.000000,0.000000,1.000000
75%,3.000000,1.000000,1.000000,2.000000,70.000000,150.000000,30.000000,4.000000,1.000000,1.000000
max,16.000000,8.500000,12.000000,22.000000,9000.000000,4280.000000,2000.000000,5000.000000,1.000000,1.000000


In [ ]:
accomodates = [1, 2, 3, 4, 5, 6, 7, 8]
bathrooms = [1.0, 1.5, 2.0]
bedrooms = [0.0, 1.0, 2.0, 3.0]
beds = [1.0, 2.0, 3.0, 4.0]
bed_type = [Real Bed, Pull-out Sofa, Futon, Couch, Airbed]
instant_bookable = [0, 1] <---boolean
minimum_nights = [1, 2, 3, 4, 5, 6, 7]
neighborhood = [Friedrichshain-Kreuzberg, Mitte, Pankow, Neukolln]
room_type = [Private room, Entire home/apt, Shared room]
wifi = [0, 1]  <---boolean

In [ ]:
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols = [
    'neighborhood',
    'room_type',
    'bed_type'
    ])

X = df.drop(['price'], axis=1)
y = df['price']

X_encoded = encoder.fit_transform(X)

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.25)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16914, 12), (5638, 12), (16914,), (5638,))

In [1]:
import pickle
filename = 'encoder.pkl'
pickle.dump(encoder, open(filename, 'wb'))

NameError: name 'encoder' is not defined

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
inputs = X_train.shape[1]

In [14]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(inputs,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Compile Model
model.compile(optimizer='adagrad', loss='mae', metrics=['mae'])

# Fit Model

model.fit(X_train, y_train, 
          validation_split=0.33,
          epochs=20
         )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 11332 samples, validate on 5582 samples
Epoch 1/20
11332/11332 [==============================] - 1s 109us/sample - loss: 41.1732 - mean_absolute_error: 41.1732 - val_loss: 29.1727 - val_mean_absolute_error: 29.1727
Epoch 2/20
11332/11332 [==============================] - 1s 72us/sample - loss: 35.4773 - mean_absolute_error: 35.4773 - val_loss: 27.6402 - val_mean_absolute_error: 27.6402
Epoch 3/20
11332/11332 [==============================] - 1s 73us/sample - loss: 34.4778 - mean_absolute_error: 34.4779 - val_loss: 27.1355 - val_mean_absolute_error: 27.1354
Epoch 4/20
11332/11332 [==============================] - 1s 68us/sample - loss: 33.8182 - mean_absolute_error: 33.8182 - val_loss: 26.5091 - val_mean_absolute_error: 26.5091
Epoch 5/20
11332/11332 [==============================] - 1s 82us/sample - loss: 33.3863 - mean_absolute_error: 33.3863 - val_loss: 

In [16]:
from sklearn.metrics import mean_absolute_error

preds = model.predict(X_test)
mean_absolute_error(preds, y_test)

30.27615238039491

In [17]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(random_state=42)


rfr.fit(X_train, y_train)
preds = rfr.predict(X_test)
mean_absolute_error(preds, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


27.36839304717985

In [18]:
rfr.feature_importances_

array([0.01966198, 0.06674001, 0.04516031, 0.04987846, 0.05114629,
       0.02157156, 0.12683606, 0.16859495, 0.19374894, 0.04558842,
       0.19179855, 0.01927447])

In [ ]:
import pickle
filename = 'model.pkl'
pickle.dump(rfr, open(filename, 'wb'))